In [31]:
import pandas as pd
from datetime import datetime as dt,timedelta
import plotly.express as px
import plotly.graph_objects as go
import plotly.colors

In [32]:
data = pd.read_csv('C:/Users/labdi/Lida Info/RFM/data/online_retail.csv')
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [33]:
data.tail()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France


In [35]:
print(data.isna().sum())

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64


In [37]:
data.dropna(subset=['CustomerID'],inplace=True)

In [38]:
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])
data['TotalAmount'] = data['Quantity'] * data['UnitPrice']

In [39]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalAmount
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [43]:
# If we are working with real data, our reference date should be today
reference_date = pd.Timestamp(dt.now().date()) # For the case that my data is updated until today
print(reference_date)

2025-12-08 00:00:00


In [45]:
reference_date = data['InvoiceDate'].max() + timedelta(days=1)  # Maximum date if column + 1; data here is dummy data
reference_date

Timestamp('2011-12-10 12:50:00')

In [47]:
rfm = data.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (reference_date - x.max()).days,
    'InvoiceNo': 'count',
    'TotalAmount':'sum'
})
print(rfm)

            InvoiceDate  InvoiceNo  TotalAmount
CustomerID                                     
12346.0             326          2         0.00
12347.0               2        182      4310.00
12348.0              75         31      1797.24
12349.0              19         73      1757.55
12350.0             310         17       334.40
...                 ...        ...          ...
18280.0             278         10       180.60
18281.0             181          7        80.82
18282.0               8         13       176.60
18283.0               4        756      2094.88
18287.0              43         70      1837.28

[4372 rows x 3 columns]


In [49]:
rfm.rename(columns={'InvoiceDate':'Recency','InvoiceNo':'Frequency','TotalAmount':'Value'},inplace=True)
print(rfm)

            Recency  Frequency    Value
CustomerID                             
12346.0         326          2     0.00
12347.0           2        182  4310.00
12348.0          75         31  1797.24
12349.0          19         73  1757.55
12350.0         310         17   334.40
...             ...        ...      ...
18280.0         278         10   180.60
18281.0         181          7    80.82
18282.0           8         13   176.60
18283.0           4        756  2094.88
18287.0          43         70  1837.28

[4372 rows x 3 columns]


In [51]:
quantiles = rfm.quantile(q=[0.25,0.5,0.75])

# Assign RFM scores
def RScore(x,p,d):
    if p == 'Recency':
        if x <= d[p][0.25]:
            return 4
        elif x <= d[p][0.50]:
            return 3
        elif x <= d[p][0.75]:
            return 2
        else:
            return 1
        
    else:  
        if x <= d[p][0.25]:
            return 1
        elif x <= d[p][0.50]:
            return 2
        elif x <= d[p][0.75]:
            return 3
        else:
            return 4 
        
rfm['R'] = rfm['Recency'].apply(RScore,args=('Recency',quantiles))
rfm['F'] = rfm['Frequency'].apply(RScore, args=('Frequency',quantiles))
rfm['M'] = rfm['Value'].apply(RScore, args=('Value',quantiles))
    
rfm.head()

,Recency,Frequency,Value,R,F,M
CustomerID,,,,,,
12346.0,326,2,0.00,1,1,1
12347.0,2,182,4310.00,4,4,4
12348.0,75,31,1797.24,2,2,4
12349.0,19,73,1757.55,3,3,4
12350.0,310,17,334.40,1,1,2


In [52]:
rfm['RFM_Segment'] = rfm['R'].astype(str) + rfm['F'].astype(str) + rfm['M'].astype(str)
rfm['RFM_Score'] = rfm[['R','F','M']].sum(axis=1)

rfm.head()

,Recency,Frequency,Value,R,F,M,RFM_Segment,RFM_Score
CustomerID,,,,,,,,
12346.0,326,2,0.00,1,1,1,111,3
12347.0,2,182,4310.00,4,4,4,444,12
12348.0,75,31,1797.24,2,2,4,224,8
12349.0,19,73,1757.55,3,3,4,334,10
12350.0,310,17,334.40,1,1,2,112,4


In [ ]:
segment_labels = ['Low-Value','Mid-Value','High-Value']

def assign_segment(score):
    if score < 5:
        return 'Low-Value'
    elif score < 9:
        return 'Mid-Value'
    else:
        return 'High-Value'
    
